In [1]:
from scipy.stats import ttest_1samp
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


**Задача:** На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

**Считываем данные**

In [3]:
df = pd.read_excel('D:\Lena\Geekbrains\Аналитика курс\АВ тестирование\Lesson 8\AB_Test_Results (2).xlsx')
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [5]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


По выручке неплохой показатель равен почти 10%. Отсутствуют отрицательные значения - значит выборка пригодна к дальнейшему анализу

In [6]:
df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [7]:
df.VARIANT_NAME.value_counts()

variant    5016
control    4984
Name: VARIANT_NAME, dtype: int64

Мы видим, что выборки по двум группам почти одинаковые

Избавимся от некорретных строк. Посчитаем, сколько типов встречается у каждого пользователя.

In [8]:
v = df.\
    groupby('USER_ID', as_index=False).\
    agg({'VARIANT_NAME': pd.Series.nunique})

In [9]:
v.head(10)

,USER_ID,VARIANT_NAME
0,2,1
1,3,2
2,4,1
3,5,1
4,6,1
5,9,1
6,10,2
7,11,1
8,12,1
9,13,1


In [10]:
more_than_one_types = v.query('VARIANT_NAME > 1')

In [11]:
df_new = df[~df.USER_ID.isin(more_than_one_types.USER_ID)].sort_values('USER_ID')

Посмотрим сколько юзеров осталось

In [12]:
df_new.shape

(6070, 3)

In [13]:
df.shape

(10000, 3)

Выборка значительно сократилась с 10 000 до 6 070, это плохой результат, но данных по-прежнему достаточно для проведения анализа. Можно сделать вывод, что выполнены неверные настройки отборки пользователей для проведения теста. Рекомендованно перенастроить.

Проверим размерность основной и тестовой групп.

In [14]:
control = df_new.query('VARIANT_NAME == "control"')

test = df_new.query('VARIANT_NAME == "variant"')

In [15]:
len(control)

3026

In [16]:
len(test)

3044

Мы видим, что по-прежнему распределение по группам почти одинаковое

In [17]:
from scipy.stats import shapiro

In [18]:
alpha = 0.05

st = shapiro(df.REVENUE)
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



Распределение не нормальное,  используем t_test для дальнейшего анализа

In [19]:
import scipy.stats as stats

In [20]:
stats.ttest_ind(control.REVENUE.values, test.REVENUE.values, equal_var = False)

Ttest_indResult(statistic=1.403762252371675, pvalue=0.1604844017690842)

По полученным данным из проведенного анализа я не рекомендую выкатывать новые изменения так как статистических различий по выручке нет. Но для более точного результата проведем расчет мощности теста

In [21]:
C_mean = control.REVENUE.values.mean()

T_mean = test.REVENUE.values.mean()


C_std = control.REVENUE.values.std()

T_std = test.REVENUE.values.std()

In [22]:
print(len(control.REVENUE.values), len(test.REVENUE.values))

3026 3044


In [23]:
n =  len(control.REVENUE.values)

In [24]:
S = np.sqrt((T_std**2 + C_std **2)/ 2)

In [25]:
effect =float((T_mean-C_mean)/ S) 

In [26]:
from statsmodels.stats.power import TTestIndPower
from statsmodels.stats.proportion import proportion_effectsize

In [27]:
alpha = 0.05

# perform power analysis
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=None,
nobs1=n, ratio=1.0, alpha=alpha)

result

0.28936707257547545

По полученным данным мы видим что мощность теста мала, поэтому я рекомендую провети А/В тест заново увеличив мощность, предварительно исправив ошибки в сплитовании юзеров